In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import cv2
import timm

# ----------------------------------------
# 1. 设置和数据加载部分 (保持不变)
# ----------------------------------------
# (此处省略与上次相同的 Dataset 定义和数据加载代码，以保持简洁)
patches_folder = r"C:\Users\Alpaca_YT\pythonSet\lung_slices_dataset\lung_slice_xy"
output_model_dir = "Train_SwinUnet_UP_newlung" # 使用新的文件夹以避免混淆
os.makedirs(output_model_dir, exist_ok=True)
num_epochs = 15
batch_size = 8
learning_rate = 1e-4

all_fns = sorted([f for f in os.listdir(patches_folder) if f.lower().endswith(".jpg")])
all_indices = list(range(len(all_fns)))
train_idxs, val_idxs = train_test_split(all_indices, test_size=0.25, random_state=42)

class RotLowHighDataset(Dataset):
    def __init__(self, patches_folder, indices, all_fns_list, transform=None):
        super().__init__()
        self.patches_folder = patches_folder
        self.transform = transform or transforms.ToTensor()
        self.fns = [all_fns_list[i] for i in indices]
    def __len__(self): return len(self.fns) * 2
    def __getitem__(self, idx):
        img_idx, rot_flag = idx // 2, idx % 2
        fn = self.fns[img_idx]
        img_path = os.path.join(self.patches_folder, fn)
        arr = np.array(Image.open(img_path).convert("L"))
        if rot_flag == 1: arr = np.rot90(arr, k=1)
        down_arr = cv2.resize(arr, (256, 32), interpolation=cv2.INTER_AREA)
        up_img = cv2.resize(down_arr, (256, 256), interpolation=cv2.INTER_LINEAR)
        inp_t = self.transform(Image.fromarray(up_img))
        tgt_t = self.transform(Image.fromarray(arr))
        return inp_t, tgt_t

class PlainLowHighDataset(Dataset):
    def __init__(self, patches_folder, indices, all_fns_list, transform=None):
        super().__init__()
        self.patches_folder = patches_folder
        self.transform = transform or transforms.ToTensor()
        self.fns = [all_fns_list[i] for i in indices]
    def __len__(self): return len(self.fns)
    def __getitem__(self, idx):
        fn = self.fns[idx]
        img_path = os.path.join(self.patches_folder, fn)
        arr = np.array(Image.open(img_path).convert("L"))
        down_arr = cv2.resize(arr, (256, 32), interpolation=cv2.INTER_AREA)
        up_img = cv2.resize(down_arr, (256, 256), interpolation=cv2.INTER_LINEAR)
        inp_t = self.transform(Image.fromarray(up_img))
        tgt_t = self.transform(Image.fromarray(arr))
        return inp_t, tgt_t


# ----------------------------------------
# 2. 定义修正后的 SwinUnet 模型
# ----------------------------------------
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, 1, 1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, 1, 1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.net(x)

class UpBlockPixelShuffle(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch * 4, 3, 1, 1)
        self.pixel_shuffle = nn.PixelShuffle(2)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x): return self.relu(self.pixel_shuffle(self.conv(x)))

class SwinUnet(nn.Module):
    def __init__(self, in_ch: int = 1, out_ch: int = 1, pretrained: bool = True):
        super().__init__()
        self.in_ch = in_ch

        # --- 主要变更点 1: 更换为Swin-Base模型 ---
        self.encoder = timm.create_model(
            'swin_base_patch4_window7_224',  # <-- 从 'swin_tiny' 升级到 'swin_base'
            pretrained=pretrained,
            features_only=True,
            in_chans=3,  # Swin Transformer 预训练模型通常需要3通道输入
            img_size=256
        )

        # 获取Swin-Base模型在每个阶段输出的通道数
        # Swin-Base的通道数通常为: [128, 256, 512, 1024]
        encoder_channels: List[int] = self.encoder.feature_info.channels()

        # --- 主要变更点 2: 调整解码器以匹配Swin-Base的通道数 ---
        # 调整UpBlock和DoubleConv的输入/输出通道，以匹配encoder_channels
        self.up4 = UpBlockPixelShuffle(encoder_channels[3], 512) # 1024 -> 512
        self.dec4 = DoubleConv(encoder_channels[2] + 512, 512)   # 512 + 512 -> 512

        self.up3 = UpBlockPixelShuffle(512, 256)
        self.dec3 = DoubleConv(encoder_channels[1] + 256, 256)   # 256 + 256 -> 256

        self.up2 = UpBlockPixelShuffle(256, 128)
        self.dec2 = DoubleConv(encoder_channels[0] + 128, 128)   # 128 + 128 -> 128

        self.up1 = UpBlockPixelShuffle(128, 64)
        # 注意: 这里的DoubleConv输入通道数也需要调整
        self.dec1 = DoubleConv(in_ch + 64, 64) # 假设最终与单通道原始输入拼接

        self.outc = nn.Conv2d(64, out_ch, kernel_size=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_orig = x
        # 如果输入是单通道，则复制为3通道以适应预训练的编码器
        if self.in_ch == 1 and x.size(1) == 1:
            x = x.repeat(1, 3, 1, 1)

        features = self.encoder(x)
        # timm的features_only=True输出的特征图通常是 (B, H, W, C) 格式
        # 我们需要将其转换为 (B, C, H, W)
        e1 = features[0].permute(0, 3, 1, 2)
        e2 = features[1].permute(0, 3, 1, 2)
        e3 = features[2].permute(0, 3, 1, 2)
        e4 = features[3].permute(0, 3, 1, 2)

        # 解码器路径
        u4 = self.up4(e4)
        d4 = self.dec4(torch.cat([u4, e3], dim=1))

        u3 = self.up3(d4)
        d3 = self.dec3(torch.cat([u3, e2], dim=1))

        u2 = self.up2(d3)
        d2 = self.dec2(torch.cat([u2, e1], dim=1))

        u1 = self.up1(d2)
        
        # 最后的上采样和与原始输入的拼接
        final_up = F.interpolate(u1, scale_factor=2, mode='bilinear', align_corners=False)
        d1 = self.dec1(torch.cat([final_up, x_orig], dim=1))
        
        return self.outc(d1)

# ----------------------------------------
# 3. 准备训练 (保持不变)
# ----------------------------------------
transform = transforms.ToTensor()
train_dataset = RotLowHighDataset(patches_folder, train_idxs, all_fns, transform)
val_dataset = PlainLowHighDataset(patches_folder, val_idxs, all_fns, transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SwinUnet(in_ch=1, out_ch=1, pretrained=True).to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# ----------------------------------------
# 4. 训练循环 (保持不变)
# ----------------------------------------
# ----------------------------------------
# 准备 Dataloader, 模型, 优化器 (保持不变)
# ----------------------------------------
# (此处假设之前的代码已定义好 model, optimizer, train_loader, val_loader, device 等变量)
# criterion 仍然是基础的MSE损失
criterion = nn.MSELoss()

# ### 修改点 1：定义高频损失所需组件 ###
# 定义拉普拉斯卷积核，并将其发送到正确的设备
laplacian_kernel = torch.tensor(
    [[0.0, -1.0, 0.0],
     [-1.0, 4.0, -1.0],
     [0.0, -1.0, 0.0]],
    device=device, dtype=torch.float32
).view(1, 1, 3, 3)

# 定义高频损失的权重
lambda_hf = 0.5

# 定义高频损失函数
def high_freq_loss(pred, target):
    """计算预测和目标之间高频分量的MSE损失"""
    pred_lap = F.conv2d(pred, laplacian_kernel, padding=1)
    tgt_lap  = F.conv2d(target, laplacian_kernel, padding=1)
    return F.mse_loss(pred_lap, tgt_lap)


# ----------------------------------------
# 训练循环 (修改损失计算部分)
# ----------------------------------------
print(f"开始在 {device} 上训练最终修正版的 SwinUnet (使用高频加权损失)...")
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0
    for inp, tgt in train_loader:
        inp, tgt = inp.to(device), tgt.to(device)
        optimizer.zero_grad()
        out = model(inp)
        
        # ### 修改点 2：计算复合损失 ###
        mse_train = criterion(out, tgt)
        hf_train = high_freq_loss(out, tgt)
        loss = mse_train + lambda_hf * hf_train # 总损失 = MSE + λ * 高频损失
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inp.size(0)
    avg_train_loss = train_loss / len(train_dataset)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inp_v, tgt_v in val_loader:
            inp_v, tgt_v = inp_v.to(device), tgt_v.to(device)
            out_v = model(inp_v)
            
            # ### 修改点 3：在验证时也使用相同的复合损失 ###
            mse_val = criterion(out_v, tgt_v)
            hf_val = high_freq_loss(out_v, tgt_v)
            loss_v = mse_val + lambda_hf * hf_val

            val_loss += loss_v.item() * inp_v.size(0)
    avg_val_loss = val_loss / len(val_dataset)

    print(f"Epoch {epoch:02d}/{num_epochs} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")

    # 保存模型权重时，可以加上标识，例如 "hf" 代表 high-frequency
    ckpt_path = os.path.join(output_model_dir, f"SwinUnet_v2_hf_epoch{epoch:02d}.pth")
    torch.save(model.state_dict(), ckpt_path)

print(f"\n训练完毕，所有模型已保存在 '{output_model_dir}/'")


C:\Users\Alpaca_YT\anaconda3\envs\mphy0041-cw2-pt\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alpaca_YT\.cache\huggingface\hub\models--timm--swin_base_patch4_window7_224.ms_in22k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


开始在 cuda 上训练最终修正版的 SwinUnet (使用高频加权损失)...
Epoch 01/15 | Train Loss: 0.035127 | Val Loss: 0.016123
Epoch 02/15 | Train Loss: 0.016485 | Val Loss: 0.015261
Epoch 03/15 | Train Loss: 0.015985 | Val Loss: 0.015000
Epoch 04/15 | Train Loss: 0.015907 | Val Loss: 0.014712
Epoch 05/15 | Train Loss: 0.015558 | Val Loss: 0.014847
Epoch 06/15 | Train Loss: 0.015098 | Val Loss: 0.014686
Epoch 07/15 | Train Loss: 0.014871 | Val Loss: 0.014656
Epoch 08/15 | Train Loss: 0.014888 | Val Loss: 0.014410
Epoch 09/15 | Train Loss: 0.014616 | Val Loss: 0.014365
Epoch 10/15 | Train Loss: 0.014673 | Val Loss: 0.014398
Epoch 11/15 | Train Loss: 0.014506 | Val Loss: 0.014172
Epoch 12/15 | Train Loss: 0.014388 | Val Loss: 0.014119
Epoch 13/15 | Train Loss: 0.014222 | Val Loss: 0.014093
Epoch 14/15 | Train Loss: 0.014208 | Val Loss: 0.014308
Epoch 15/15 | Train Loss: 0.014201 | Val Loss: 0.013878

训练完毕，所有模型已保存在 'Train_SwinUnet_UP_newlung/'


In [4]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.metrics import peak_signal_noise_ratio as psnr_metric
from skimage.metrics import structural_similarity as ssim_metric
import cv2
import timm

# ----------------------------------------
# 1. 定义与训练时完全一致的模型和 Dataset 类
# ----------------------------------------
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.net(x)

class UpBlockPixelShuffle(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch * 4, 3, 1, 1)
        self.pixel_shuffle = nn.PixelShuffle(2)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x): return self.relu(self.pixel_shuffle(self.conv(x)))

class SwinUnet(nn.Module):
    def __init__(self, in_ch: int = 1, out_ch: int = 1, pretrained: bool = True):
        super().__init__()
        self.in_ch = in_ch

        # --- 主要变更点 1: 更换为Swin-Base模型 ---
        self.encoder = timm.create_model(
            'swin_base_patch4_window7_224',  # <-- 从 'swin_tiny' 升级到 'swin_base'
            pretrained=pretrained,
            features_only=True,
            in_chans=3,  # Swin Transformer 预训练模型通常需要3通道输入
            img_size=256
        )

        # 获取Swin-Base模型在每个阶段输出的通道数
        # Swin-Base的通道数通常为: [128, 256, 512, 1024]
        encoder_channels: List[int] = self.encoder.feature_info.channels()

        # --- 主要变更点 2: 调整解码器以匹配Swin-Base的通道数 ---
        # 调整UpBlock和DoubleConv的输入/输出通道，以匹配encoder_channels
        self.up4 = UpBlockPixelShuffle(encoder_channels[3], 512) # 1024 -> 512
        self.dec4 = DoubleConv(encoder_channels[2] + 512, 512)   # 512 + 512 -> 512

        self.up3 = UpBlockPixelShuffle(512, 256)
        self.dec3 = DoubleConv(encoder_channels[1] + 256, 256)   # 256 + 256 -> 256

        self.up2 = UpBlockPixelShuffle(256, 128)
        self.dec2 = DoubleConv(encoder_channels[0] + 128, 128)   # 128 + 128 -> 128

        self.up1 = UpBlockPixelShuffle(128, 64)
        # 注意: 这里的DoubleConv输入通道数也需要调整
        self.dec1 = DoubleConv(in_ch + 64, 64) # 假设最终与单通道原始输入拼接

        self.outc = nn.Conv2d(64, out_ch, kernel_size=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_orig = x
        # 如果输入是单通道，则复制为3通道以适应预训练的编码器
        if self.in_ch == 1 and x.size(1) == 1:
            x = x.repeat(1, 3, 1, 1)

        features = self.encoder(x)
        # timm的features_only=True输出的特征图通常是 (B, H, W, C) 格式
        # 我们需要将其转换为 (B, C, H, W)
        e1 = features[0].permute(0, 3, 1, 2)
        e2 = features[1].permute(0, 3, 1, 2)
        e3 = features[2].permute(0, 3, 1, 2)
        e4 = features[3].permute(0, 3, 1, 2)

        # 解码器路径
        u4 = self.up4(e4)
        d4 = self.dec4(torch.cat([u4, e3], dim=1))

        u3 = self.up3(d4)
        d3 = self.dec3(torch.cat([u3, e2], dim=1))

        u2 = self.up2(d3)
        d2 = self.dec2(torch.cat([u2, e1], dim=1))

        u1 = self.up1(d2)
        
        # 最后的上采样和与原始输入的拼接
        final_up = F.interpolate(u1, scale_factor=2, mode='bilinear', align_corners=False)
        d1 = self.dec1(torch.cat([final_up, x_orig], dim=1))
        
        return self.outc(d1)


# ### 修改点 1：让 Dataset 返回文件名 ###
class PlainLowHighDataset(Dataset):
    def __init__(self, patches_folder, indices, all_fns_list, transform=None):
        super().__init__()
        self.patches_folder, self.transform = patches_folder, transform or transforms.ToTensor()
        self.fns = [all_fns_list[i] for i in indices]
    def __len__(self): return len(self.fns)
    def __getitem__(self, idx):
        fn = self.fns[idx]
        img_path = os.path.join(self.patches_folder, fn)
        arr = np.array(Image.open(img_path).convert("L"))
        down_arr = cv2.resize(arr, (256, 32), interpolation=cv2.INTER_AREA)
        up_img = cv2.resize(down_arr, (256, 256), interpolation=cv2.INTER_LINEAR)
        inp_t = self.transform(Image.fromarray(up_img))
        tgt_t = self.transform(Image.fromarray(arr))
        return inp_t, tgt_t, fn # 返回文件名

# ----------------------------------------
# 2. 设置路径和参数
# ----------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.ToTensor()
# ### 新增：定义保存图像的文件夹和数量 ###
RECONSTRUCTION_DIR = "lung_SwinNet_reconstructions"
IMAGES_TO_SAVE = 5
os.makedirs(RECONSTRUCTION_DIR, exist_ok=True)

# 准备 Dataloaders (与之前相同)
val_patches_folder = r"C:\Users\Alpaca_YT\pythonSet\lung_slices_dataset\lung_slice_xy"
all_val_fns = sorted([f for f in os.listdir(val_patches_folder) if f.lower().endswith((".jpg", ".png"))])
all_val_indices = list(range(len(all_val_fns)))
_, val_idxs = train_test_split(all_val_indices, test_size=0.25, random_state=42)
val_dataset = PlainLowHighDataset(val_patches_folder, val_idxs, all_val_fns, transform)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

test_patches_folder = r"C:\Users\Alpaca_YT\pythonSet\lung_slices_dataset\lung_slice_yz"
all_test_fns = sorted([f for f in os.listdir(test_patches_folder) if f.lower().endswith((".jpg", ".png"))])
all_test_indices = list(range(len(all_test_fns)))
test_dataset = PlainLowHighDataset(test_patches_folder, all_test_indices, all_test_fns, transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# ----------------------------------------
# 3. 加载模型权重 (与之前相同)
# ----------------------------------------
model = SwinUnet(in_ch=1, out_ch=1, pretrained=False).to(device)
checkpoint_path = r"Train_SwinUnet_UP_newlung/SwinUnet_v2_hf_epoch15.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()
print(f"成功加载模型权重: {checkpoint_path}")

# ----------------------------------------
# 4. 定义评估函数 (增加保存逻辑)
# ----------------------------------------
def tensor_to_image(tensor):
    """将 [0,1] 范围的Tensor转换为可保存的PIL Image对象"""
    arr = tensor.clamp(0, 1).cpu().squeeze().numpy()
    return Image.fromarray((arr * 255).round().astype(np.uint8))

# ### 修改点 2：为函数增加保存功能相关的参数 ###
def evaluate_model(loader, model_to_test, output_dir=None, save_count=0):
    total_psnr, total_ssim, count = 0.0, 0.0, 0
    with torch.no_grad():
        # ### 修改点 3：使用 enumerate 以获取索引 ###
        for i, (inp_t, tgt_t, fn) in enumerate(loader):
            # 解包文件名元组（如果dataloader返回的是元组）
            fn = fn[0] if isinstance(fn, (list, tuple)) else fn

            inp_t, tgt_t = inp_t.to(device), tgt_t.to(device)
            out_t = model_to_test(inp_t)
            
            pred_np = out_t.clamp(0, 1).cpu().squeeze().numpy()
            target_np = tgt_t.cpu().squeeze().numpy()
            
            total_psnr += psnr_metric(target_np, pred_np, data_range=1.0)
            total_ssim += ssim_metric(target_np, pred_np, data_range=1.0, channel_axis=None)
            count += 1
            
            # ### 修改点 4：保存指定数量的图像 ###
            if output_dir is not None and i < save_count:
                base_name = os.path.splitext(fn)[0]
                # 保存三种图像以供对比
                tensor_to_image(inp_t).save(os.path.join(output_dir, f"{base_name}_01_input.png"))
                tensor_to_image(out_t).save(os.path.join(output_dir, f"{base_name}_02_reconstructed.png"))
                tensor_to_image(tgt_t).save(os.path.join(output_dir, f"{base_name}_03_ground_truth.png"))

    return total_psnr / count, total_ssim / count

# ----------------------------------------
# 5. 执行评估并打印结果
# ----------------------------------------
print(f"\n在 {device} 上评估模型...")

# 评估验证集，不保存图像
val_psnr, val_ssim = evaluate_model(val_loader, model)
print(f"\n-> 验证集结果:")
print(f"   - 平均 PSNR: {val_psnr:.4f} dB")
print(f"   - 平均 SSIM: {val_ssim:.4f}")

# ### 修改点 5：评估测试集，并传入保存图像的参数 ###
test_psnr, test_ssim = evaluate_model(test_loader, model, output_dir=RECONSTRUCTION_DIR, save_count=IMAGES_TO_SAVE)
print(f"\n-> 测试集结果:")
print(f"   - 平均 PSNR: {test_psnr:.4f} dB")
print(f"   - 平均 SSIM: {test_ssim:.4f}")
print(f"\n已保存 {IMAGES_TO_SAVE} 组测试图像到 '{RECONSTRUCTION_DIR}/' 文件夹中供查看。")

C:\Users\Alpaca_YT\AppData\Local\Temp\ipykernel_37200\870280465.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path, map_l

成功加载模型权重: Train_SwinUnet_UP_newlung/SwinUnet_v2_hf_epoch15.pth

在 cuda 上评估模型...

-> 验证集结果:
   - 平均 PSNR: 24.6470 dB
   - 平均 SSIM: 0.6209

-> 测试集结果:
   - 平均 PSNR: 24.1537 dB
   - 平均 SSIM: 0.5899

已保存 5 组测试图像到 'lung_SwinNet_reconstructions/' 文件夹中供查看。


In [5]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def calculate_metrics(image_true, image_test):
    """
    计算两张图像之间的 PSNR 和 SSIM.

    参数:
    image_true (np.ndarray): 真实图像 (参考图像).
    image_test (np.ndarray): 测试图像 (待评估图像).

    返回:
    tuple: (psnr_value, ssim_value).
    """
    # 1. 检查图像尺寸是否一致
    if image_true.shape != image_test.shape:
        raise ValueError("输入图像的尺寸必须完全相同。")

    # 2. 计算 PSNR
    # data_range 是图像像素值的可能范围。对于8位图像，通常是 255。
    psnr_value = psnr(image_true, image_test, data_range=255)

    # 3. 计算 SSIM
    # SSIM 通常在灰度图上计算。如果图像是彩色的，我们先将其转换为灰度图。
    # scikit-image 的 ssim 函数也可以处理多通道图像，只需设置 channel_axis=-1
    # 但最常见的做法还是先转为灰度图。
    if image_true.ndim == 3:
        gray_true = cv2.cvtColor(image_true, cv2.COLOR_BGR2GRAY)
        gray_test = cv2.cvtColor(image_test, cv2.COLOR_BGR2GRAY)
        ssim_value = ssim(gray_true, gray_test, data_range=255)
    else:
        # 如果已经是灰度图
        ssim_value = ssim(image_true, image_test, data_range=255)
        
    return psnr_value, ssim_value



path_true = r"test1/Cropped_img01576.png"
path_test = r"test1/I3-1576.png"

try:
    # 使用 OpenCV 读取图片
    img_true = cv2.imread(path_true)
    img_test = cv2.imread(path_test)

    # 检查图片是否成功加载
    if img_true is None or img_test is None:
        raise IOError("无法加载一张或多张图片，请检查路径是否正确。")
    
    # 如果图片尺寸不同，可以先将其调整为一致（例如，调整为第一张图片的尺寸）
    if img_true.shape != img_test.shape:
        print(f"警告：图片尺寸不同，将第二张图片调整为与第一张相同 ({img_true.shape})。")
        img_test = cv2.resize(img_test, (img_true.shape[1], img_true.shape[0]))

    psnr_file, ssim_file = calculate_metrics(img_true, img_test)
    print(f"文件图像 PSNR: {psnr_file:.2f} dB")
    print(f"文件图像 SSIM: {ssim_file:.4f}")

except (IOError, ValueError) as e:
    print(f"处理文件时发生错误: {e}")
    print("请将 'path/to/your/...' 替换为您的真实图片路径后重试。")

文件图像 PSNR: 23.85 dB
文件图像 SSIM: 0.5130
